In [4]:
import pandas as pd

def translate_team(team_name):
    # Check if translation exists, otherwise return the original name
    return translation_dict.get(team_name, f"Unknown ({team_name})")

Head_path = 'D:\Projects\Other_exp\Jupyter\orie4741-fp\prediction_data/'
# Step 1: Load the prediction CSV
prediction_df = pd.read_csv(Head_path+'prediction.csv')

# Step 2: Load the translation CSV (assuming it has 'Chinese' and 'English' columns)
translation_df = pd.read_csv(Head_path+'translation.csv', encoding='utf-8')

# Step 3: Create a dictionary for mapping Chinese team names to English
translation_dict = pd.Series(translation_df['English'].values, index=translation_df['Chinese'].values).to_dict()

# Step 3: Create a dictionary for mapping Chinese aHh to Math
translation_dict_math = pd.Series(translation_df['Math_Hh'].values, index=translation_df['Chinese_Hh'].values).to_dict()

# Step 4: Replace Chinese team names with English in HomeTeam and AwayTeam columns
prediction_df['HomeTeam'] = prediction_df['HomeTeam'].map(translation_dict).fillna(prediction_df['HomeTeam'])
prediction_df['AwayTeam'] = prediction_df['AwayTeam'].map(translation_dict).fillna(prediction_df['AwayTeam'])
prediction_df['AHh'] = prediction_df['AHh'].map(translation_dict_math).fillna(prediction_df['AHh'])
# prediction_df['HomeTeam'] = prediction_df['HomeTeam'].apply(translate_team)
# prediction_df['AwayTeam'] = prediction_df['AwayTeam'].apply(translate_team)

# Step 5: Save the updated dataframe (optional)
prediction_df.to_csv(Head_path+'updated_prediction.csv', index=False, encoding='utf-8-sig')
print(prediction_df)

    Unnamed: 0  Div       Date     HomeTeam       AwayTeam  B365H  B365D  \
0            4  SP1  2025/1/24   Las Palmas        Osasuna   2.45   3.10   
1            5   E0  2025/1/25  Bournemouth  Nott'm Forest   2.05   3.70   
2            6   E0  2025/1/25     Brighton        Everton   1.67   3.75   
3            7   E0  2025/1/25    Liverpool        Ipswich   1.11  12.00   
4            8   E0  2025/1/25  Southampton      Newcastle   7.00   5.25   
5            9   E0  2025/1/25       Wolves        Arsenal   6.50   4.50   
6           10   E0  2025/1/25     Man City        Chelsea   1.95   4.10   
7           11   D1  2025/1/25     Augsburg     Heidenheim   1.85   3.60   
8           12   D1  2025/1/25     Dortmund  Werder Bremen   1.62   4.33   
9           13   D1  2025/1/25     Freiburg  Bayern Munich   7.50   5.75   
10          14   D1  2025/1/25        Mainz      Stuttgart   2.80   3.50   
11          15   D1  2025/1/25   RB Leipzig     Leverkusen   3.60   3.80   
12          

In [75]:
# import os
# 
# # 文件夹路径
# folder_path = 'D:\Projects\Other_exp\Jupyter\orie4741-fp\elo/'
# 
# # 获取文件夹下所有文件的名称
# file_names = []
# for root, dirs, files in os.walk(folder_path):
#     for file in files:
#         # 分离文件名和扩展名，获取文件名部分
#         file_name_without_extension = os.path.splitext(file)[0]
#         file_names.append(file_name_without_extension)
# 
# # 将文件名称写入到 txt 文件，按行存储
# with open(folder_path + 'file_names.txt', 'w', encoding='utf-8') as f:
#     for file_name in file_names:
#         f.write(file_name + '\n')
# 
# print("文件名称已成功保存到 file_names.txt")

In [5]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
import joblib
from joblib import load, dump
import time
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

In [6]:
df = pd.read_csv('prediction_data/updated_prediction.csv', low_memory = False)
df

,Unnamed: 0,Div,Date,HomeTeam,AwayTeam,B365H,B365D,B365A,AHh,B365AHH,B365AHA,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,4,SP1,2025/1/24,Las Palmas,Osasuna,2.45,3.10,3.00,-0.25,2.08,1.82,NaN,2.0,NaN,1：0 以上
1,5,E0,2025/1/25,Bournemouth,Nott'm Forest,2.05,3.70,3.40,-0.25,1.83,2.07,NaN,0.0,NaN,0：0 / 0：1
2,6,E0,2025/1/25,Brighton,Everton,1.67,3.75,5.50,-0.75,1.87,2.03,NaN,2.0,NaN,1：0/ 2：0
3,7,E0,2025/1/25,Liverpool,Ipswich,1.11,12.00,15.00,-2.50,1.87,2.03,NaN,0.0,NaN,2：0 以下
4,8,E0,2025/1/25,Southampton,Newcastle,7.00,5.25,1.40,1.25,2.03,1.87,NaN,2.0,NaN,0：1 以下
5,9,E0,2025/1/25,Wolves,Arsenal,6.50,4.50,1.45,1.25,1.83,2.07,NaN,0.0,NaN,0：2 以上
6,10,E0,2025/1/25,Man City,Chelsea,1.95,4.10,3.50,-0.50,2.01,1.89,NaN,0.0,NaN,0：0 / 0：1
7,11,D1,2025/1/25,Augsburg,Heidenheim,1.85,3.60,4.00,-0.50,1.91,1.99,NaN,2.0,NaN,1：0 以上
8,12,D1,2025/1/25,Dortmund,Werder Bremen,1.62,4.33,5.00,-1.00,2.05,1.85,NaN,2.0,NaN,2：0以上
9,13,D1,2025/1/25,Freiburg,Bayern Munich,7.50,5.75,1.33,1.50,2.00,1.90,NaN,0.0,NaN,0：2以上


In [7]:
# remove unknown columns
columns = df.columns
unknown_cols = [c for c in columns if "Unnamed" in c]
df = df.drop(columns=unknown_cols)

# remove rows with only na values
df = df.dropna(how="all")

features_to_keep = ["Div", "Date", "HomeTeam", "AwayTeam", "B365H", "B365D", "B365A",
                    "AHh", "B365AHH", "B365AHA"
                    ]
df = df[features_to_keep]

In [8]:
from os.path import exists
all_teams = set(df["HomeTeam"].unique()).union(set(df["AwayTeam"].unique()))

dfs = {}

for team in all_teams:
    if exists("elo/" + team + ".csv"):
        dfs[team] = pd.read_csv("elo/" + team + ".csv")
def find_team_elo_at(team_name, date):
    elo_df = dfs.get(team_name, None)
    if elo_df is None:
        return None
    elo_df["From"] = pd.to_datetime(elo_df["From"], format='mixed', dayfirst=True)
    elo_df["To"] = pd.to_datetime(elo_df["To"], format='mixed', dayfirst=True)
    if date > '2000-05-01':
        aa = 1
    # if date > '2024-10-01':
    #     aa = 2
    try:
        # elo = elo_df.loc[elo_df["From"] >= date]["Elo"].iloc[0]
        # print(elo_df.loc[(elo_df["From"] <= date) & (elo_df["To"] > date)])
        elo = elo_df.loc[(elo_df["From"] <= date) & (elo_df["To"] >= date)]["Elo"].iloc[0]
        return elo
    except:
        return None
    
def add_elo_feature(s):
    s["HomeTeamELO"] = find_team_elo_at(s["HomeTeam"], s["Date"])
    s["AwayTeamELO"] = find_team_elo_at(s["AwayTeam"], s["Date"])
    return s

f = np.vectorize(find_team_elo_at)

df["HomeTeamELO"] = f(df["HomeTeam"], df["Date"])
df["AwayTeamELO"] = f(df["AwayTeam"], df["Date"])

# df = df.apply(add_elo_feature, axis=1)
df.to_csv("prediction_data/updated_prediction_with_elo.csv")
df

,Div,Date,HomeTeam,AwayTeam,B365H,B365D,B365A,AHh,B365AHH,B365AHA,HomeTeamELO,AwayTeamELO
0,SP1,2025/1/24,Las Palmas,Osasuna,2.45,3.10,3.00,-0.25,2.08,1.82,1602.423584,1655.849487
1,E0,2025/1/25,Bournemouth,Nott'm Forest,2.05,3.70,3.40,-0.25,1.83,2.07,1802.067261,1785.947388
2,E0,2025/1/25,Brighton,Everton,1.67,3.75,5.50,-0.75,1.87,2.03,1779.467041,1706.522461
3,E0,2025/1/25,Liverpool,Ipswich,1.11,12.00,15.00,-2.50,1.87,2.03,2002.760498,1597.615112
4,E0,2025/1/25,Southampton,Newcastle,7.00,5.25,1.40,1.25,2.03,1.87,1551.392090,1834.406738
5,E0,2025/1/25,Wolves,Arsenal,6.50,4.50,1.45,1.25,1.83,2.07,1669.096313,1982.405396
6,E0,2025/1/25,Man City,Chelsea,1.95,4.10,3.50,-0.50,2.01,1.89,1945.827637,1859.114746
7,D1,2025/1/25,Augsburg,Heidenheim,1.85,3.60,4.00,-0.50,1.91,1.99,1609.723633,1560.995728
8,D1,2025/1/25,Dortmund,Werder Bremen,1.62,4.33,5.00,-1.00,2.05,1.85,1777.401733,1651.901489
9,D1,2025/1/25,Freiburg,Bayern Munich,7.50,5.75,1.33,1.50,2.00,1.90,1658.334595,1912.523193


In [9]:
df_noNa = df.dropna()
df_noNa.reset_index(inplace=True, drop=True)
df_noNa

,Div,Date,HomeTeam,AwayTeam,B365H,B365D,B365A,AHh,B365AHH,B365AHA,HomeTeamELO,AwayTeamELO
0,SP1,2025/1/24,Las Palmas,Osasuna,2.45,3.10,3.00,-0.25,2.08,1.82,1602.423584,1655.849487
1,E0,2025/1/25,Bournemouth,Nott'm Forest,2.05,3.70,3.40,-0.25,1.83,2.07,1802.067261,1785.947388
2,E0,2025/1/25,Brighton,Everton,1.67,3.75,5.50,-0.75,1.87,2.03,1779.467041,1706.522461
3,E0,2025/1/25,Liverpool,Ipswich,1.11,12.00,15.00,-2.50,1.87,2.03,2002.760498,1597.615112
4,E0,2025/1/25,Southampton,Newcastle,7.00,5.25,1.40,1.25,2.03,1.87,1551.392090,1834.406738
5,E0,2025/1/25,Wolves,Arsenal,6.50,4.50,1.45,1.25,1.83,2.07,1669.096313,1982.405396
6,E0,2025/1/25,Man City,Chelsea,1.95,4.10,3.50,-0.50,2.01,1.89,1945.827637,1859.114746
7,D1,2025/1/25,Augsburg,Heidenheim,1.85,3.60,4.00,-0.50,1.91,1.99,1609.723633,1560.995728
8,D1,2025/1/25,Dortmund,Werder Bremen,1.62,4.33,5.00,-1.00,2.05,1.85,1777.401733,1651.901489
9,D1,2025/1/25,Freiburg,Bayern Munich,7.50,5.75,1.33,1.50,2.00,1.90,1658.334595,1912.523193


In [10]:
# df_noNa_new = df_noNa.iloc[:3]
df_noNa_new = df_noNa

In [11]:
# 与历史数据合并，计算后续特征
history_df = pd.read_csv(Head_path+'history_data.csv')
history_df

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,B365H,B365D,B365A,WHH,WHD,WHA,AHh,B365AHH,B365AHA,HomeTeamELO,AwayTeamELO,easy_label
0,F1,2003/8/2,Auxerre,Nice,1,2,1.727,3.100,4.500,1.66,3.1,5.00,-0.75,2.050,1.850,1702.604858,1611.196045,-1
1,F1,2003/8/2,Guingamp,Marseille,0,1,2.500,2.875,2.625,2.60,3.1,2.40,0.00,1.925,1.975,1685.016113,1665.625732,-1
2,D1,2003/8/2,Hamburg,Hannover,0,3,1.571,3.500,5.000,1.57,3.5,5.00,-0.75,1.800,2.100,1718.566284,1649.805298,-1
3,D1,2003/8/2,Hertha,Werder Bremen,0,3,1.833,3.200,3.750,1.72,3.4,4.00,-0.75,2.025,1.875,1719.916748,1692.120972,-1
4,F1,2003/8/2,Lens,Le Mans,0,0,1.571,3.250,5.500,1.66,3.3,4.50,-0.75,1.900,2.000,1697.354004,1539.958130,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37290,F1,2025/1/19,St Etienne,Nantes,1,1,2.450,3.250,2.900,2.50,3.1,2.88,0.00,1.810,2.090,1518.982300,1559.531860,0
37291,D1,2025/1/19,Union Berlin,Mainz,2,1,2.550,3.200,2.880,2.60,3.1,2.88,0.00,1.840,2.060,1590.621826,1710.893066,1
37292,SP1,2025/1/19,Valencia,Sociedad,1,0,3.100,3.000,2.550,3.20,3.0,2.45,0.25,1.730,2.080,1613.392090,1742.600586,1
37293,I1,2025/1/19,Verona,Lazio,0,3,5.000,3.500,1.750,4.50,3.6,1.75,0.75,1.870,2.030,1620.682861,1810.438232,-1


In [12]:
# 合并两个 DataFrame，重置索引
df_noNa_comb = pd.concat([history_df, df_noNa_new], ignore_index=True)
df_noNa_comb

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,B365H,B365D,B365A,WHH,WHD,WHA,AHh,B365AHH,B365AHA,HomeTeamELO,AwayTeamELO,easy_label
0,F1,2003/8/2,Auxerre,Nice,1.0,2.0,1.727,3.100,4.500,1.66,3.1,5.0,-0.75,2.050,1.850,1702.604858,1611.196045,-1.0
1,F1,2003/8/2,Guingamp,Marseille,0.0,1.0,2.500,2.875,2.625,2.60,3.1,2.4,0.00,1.925,1.975,1685.016113,1665.625732,-1.0
2,D1,2003/8/2,Hamburg,Hannover,0.0,3.0,1.571,3.500,5.000,1.57,3.5,5.0,-0.75,1.800,2.100,1718.566284,1649.805298,-1.0
3,D1,2003/8/2,Hertha,Werder Bremen,0.0,3.0,1.833,3.200,3.750,1.72,3.4,4.0,-0.75,2.025,1.875,1719.916748,1692.120972,-1.0
4,F1,2003/8/2,Lens,Le Mans,0.0,0.0,1.571,3.250,5.500,1.66,3.3,4.5,-0.75,1.900,2.000,1697.354004,1539.958130,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37308,SP1,2025/1/25,Ath Madrid,Villarreal,NaN,NaN,1.670,4.000,5.000,NaN,NaN,NaN,-0.75,1.890,2.010,1854.542480,1730.981079,NaN
37309,SP1,2025/1/25,Valladolid,Real Madrid,NaN,NaN,10.000,6.000,1.270,NaN,NaN,NaN,1.75,1.900,2.000,1551.154175,1954.102173,NaN
37310,SP1,2025/1/25,Mallorca,Betis,NaN,NaN,2.350,3.100,3.300,NaN,NaN,NaN,-0.25,2.000,1.900,1663.129639,1654.958984,NaN
37311,I1,2025/1/25,Como,Atalanta,NaN,NaN,4.200,3.600,1.850,NaN,NaN,NaN,0.50,2.020,1.880,1590.385498,1901.152222,NaN


In [13]:
# # 加载编码器
# div_encoder = joblib.load('div_encoder.pkl')
# home_encoder = joblib.load('home_encoder.pkl')
# away_encoder = joblib.load('away_encoder.pkl')
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

div_encoder = OneHotEncoder()
home_encoder = OneHotEncoder()
away_encoder = OneHotEncoder()
# 你可以使用加载的编码器进行转换
onehot_div = div_encoder.fit_transform(df_noNa_comb.Div.values.reshape(-1,1)).toarray().astype(int)
onehot_div_df = pd.DataFrame(onehot_div, columns = ["Div "+str(int(i)) for i in range(onehot_div.shape[1])])

onehot_home = home_encoder.fit_transform(df_noNa_comb.HomeTeam.values.reshape(-1,1)).toarray().astype(int)
onehot_home_df = pd.DataFrame(onehot_home, columns = ['HomeTeam ' + str(int(i)) for i in np.arange(onehot_home.shape[1])])

onehot_away = away_encoder.fit_transform(df_noNa_comb.AwayTeam.values.reshape(-1,1)).toarray().astype(int)
onehot_away_df = pd.DataFrame(onehot_away, columns = ['AwayTeam ' + str(int(i)) for i in np.arange(onehot_away.shape[1])])

df_noNa_div = pd.concat([df_noNa_comb, onehot_div_df, onehot_home_df, onehot_away_df], axis = 1)
df_noNa_div.drop(columns = ['Div'], inplace = True)
df_noNa_div

,Date,HomeTeam,AwayTeam,FTHG,FTAG,B365H,B365D,B365A,WHH,WHD,...,AwayTeam 197,AwayTeam 198,AwayTeam 199,AwayTeam 200,AwayTeam 201,AwayTeam 202,AwayTeam 203,AwayTeam 204,AwayTeam 205,AwayTeam 206
0,2003/8/2,Auxerre,Nice,1.0,2.0,1.727,3.100,4.500,1.66,3.1,...,0,0,0,0,0,0,0,0,0,0
1,2003/8/2,Guingamp,Marseille,0.0,1.0,2.500,2.875,2.625,2.60,3.1,...,0,0,0,0,0,0,0,0,0,0
2,2003/8/2,Hamburg,Hannover,0.0,3.0,1.571,3.500,5.000,1.57,3.5,...,0,0,0,0,0,0,0,0,0,0
3,2003/8/2,Hertha,Werder Bremen,0.0,3.0,1.833,3.200,3.750,1.72,3.4,...,0,0,1,0,0,0,0,0,0,0
4,2003/8/2,Lens,Le Mans,0.0,0.0,1.571,3.250,5.500,1.66,3.3,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37308,2025/1/25,Ath Madrid,Villarreal,NaN,NaN,1.670,4.000,5.000,NaN,NaN,...,1,0,0,0,0,0,0,0,0,0
37309,2025/1/25,Valladolid,Real Madrid,NaN,NaN,10.000,6.000,1.270,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
37310,2025/1/25,Mallorca,Betis,NaN,NaN,2.350,3.100,3.300,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
37311,2025/1/25,Como,Atalanta,NaN,NaN,4.200,3.600,1.850,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


In [14]:
target_encoder = LabelEncoder()
df_noNa_div['Result'] = target_encoder.fit_transform(df_noNa_div.easy_label) 

In [15]:
df_noNa_div['Year'] = pd.DatetimeIndex(df_noNa_div.Date).year

df_noNa_div['Month'] = pd.DatetimeIndex(df_noNa_div.Date).month
df_noNa_div['Sin_Month'] = np.sin(2*np.pi*df_noNa_div.Month/12)
df_noNa_div['Cos_Month'] = np.cos(2*np.pi*df_noNa_div.Month/12)

df_noNa_div['DayofYear'] = pd.DatetimeIndex(df_noNa_div.Date).dayofyear
df_noNa_div['Sin_Day'] = np.sin(2*np.pi*df_noNa_div.DayofYear/365)
df_noNa_div['Cos_Day'] = np.cos(2*np.pi*df_noNa_div.DayofYear/365)

# 注意 inplace是在原始frame修改，返回值是Nonetype
# learning_df = learning_df_div.drop(columns = ['Date','Month'], inplace = True)
df_noNa_div_date = df_noNa_div.drop(columns = ['Date','Month'])
df_noNa_div_date

,HomeTeam,AwayTeam,FTHG,FTAG,B365H,B365D,B365A,WHH,WHD,WHA,...,AwayTeam 204,AwayTeam 205,AwayTeam 206,Result,Year,Sin_Month,Cos_Month,DayofYear,Sin_Day,Cos_Day
0,Auxerre,Nice,1.0,2.0,1.727,3.100,4.500,1.66,3.1,5.0,...,0,0,0,0,2003,-0.866025,-0.500000,214,-0.516062,-0.856551
1,Guingamp,Marseille,0.0,1.0,2.500,2.875,2.625,2.60,3.1,2.4,...,0,0,0,0,2003,-0.866025,-0.500000,214,-0.516062,-0.856551
2,Hamburg,Hannover,0.0,3.0,1.571,3.500,5.000,1.57,3.5,5.0,...,0,0,0,0,2003,-0.866025,-0.500000,214,-0.516062,-0.856551
3,Hertha,Werder Bremen,0.0,3.0,1.833,3.200,3.750,1.72,3.4,4.0,...,0,0,0,0,2003,-0.866025,-0.500000,214,-0.516062,-0.856551
4,Lens,Le Mans,0.0,0.0,1.571,3.250,5.500,1.66,3.3,4.5,...,0,0,0,0,2003,-0.866025,-0.500000,214,-0.516062,-0.856551
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37308,Ath Madrid,Villarreal,NaN,NaN,1.670,4.000,5.000,NaN,NaN,NaN,...,0,0,0,3,2025,0.500000,0.866025,25,0.417194,0.908818
37309,Valladolid,Real Madrid,NaN,NaN,10.000,6.000,1.270,NaN,NaN,NaN,...,0,0,0,3,2025,0.500000,0.866025,25,0.417194,0.908818
37310,Mallorca,Betis,NaN,NaN,2.350,3.100,3.300,NaN,NaN,NaN,...,0,0,0,3,2025,0.500000,0.866025,25,0.417194,0.908818
37311,Como,Atalanta,NaN,NaN,4.200,3.600,1.850,NaN,NaN,NaN,...,0,0,0,3,2025,0.500000,0.866025,25,0.417194,0.908818


In [16]:
# 定义一个函数来计算两队之间上一场比赛的结果
def compute_last_matches(df):
    
    unique_matchups = list(set((list(zip(df.HomeTeam, df.AwayTeam)))))
    df['Last Match Result'] = np.nan
    for home, away in unique_matchups:
        matchup_df = df[(df.HomeTeam == home) & (df.AwayTeam == away)]
        # 使用 shift(1) 方法将 FTR（全场比赛结果）列中的数据向下移动一行，这样每行的 last_match_result 将对应于这两队之前的一场比赛的结果。fill_value='Na' 确保了数据移动后空出的位置填充为 'Na'。
        # last_match_result = matchup_df.FTR.shift(1, fill_value='Na')
        last_match_result = matchup_df.easy_label.shift(1, fill_value=3)
        df.loc[matchup_df.index, 'Last Match Result'] = last_match_result
        
    lmr_encoder = LabelEncoder()
    df['Last Match Result'] = lmr_encoder.fit_transform(df['Last Match Result'])
    df.drop(columns = ['easy_label'], inplace = True)
    return df
df_noNa_div_date_lastMatch = compute_last_matches(df_noNa_div_date)
df_noNa_div_date_lastMatch

,HomeTeam,AwayTeam,FTHG,FTAG,B365H,B365D,B365A,WHH,WHD,WHA,...,AwayTeam 205,AwayTeam 206,Result,Year,Sin_Month,Cos_Month,DayofYear,Sin_Day,Cos_Day,Last Match Result
0,Auxerre,Nice,1.0,2.0,1.727,3.100,4.500,1.66,3.1,5.0,...,0,0,0,2003,-0.866025,-0.500000,214,-0.516062,-0.856551,3
1,Guingamp,Marseille,0.0,1.0,2.500,2.875,2.625,2.60,3.1,2.4,...,0,0,0,2003,-0.866025,-0.500000,214,-0.516062,-0.856551,3
2,Hamburg,Hannover,0.0,3.0,1.571,3.500,5.000,1.57,3.5,5.0,...,0,0,0,2003,-0.866025,-0.500000,214,-0.516062,-0.856551,3
3,Hertha,Werder Bremen,0.0,3.0,1.833,3.200,3.750,1.72,3.4,4.0,...,0,0,0,2003,-0.866025,-0.500000,214,-0.516062,-0.856551,3
4,Lens,Le Mans,0.0,0.0,1.571,3.250,5.500,1.66,3.3,4.5,...,0,0,0,2003,-0.866025,-0.500000,214,-0.516062,-0.856551,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37308,Ath Madrid,Villarreal,NaN,NaN,1.670,4.000,5.000,NaN,NaN,NaN,...,0,0,3,2025,0.500000,0.866025,25,0.417194,0.908818,2
37309,Valladolid,Real Madrid,NaN,NaN,10.000,6.000,1.270,NaN,NaN,NaN,...,0,0,3,2025,0.500000,0.866025,25,0.417194,0.908818,0
37310,Mallorca,Betis,NaN,NaN,2.350,3.100,3.300,NaN,NaN,NaN,...,0,0,3,2025,0.500000,0.866025,25,0.417194,0.908818,0
37311,Como,Atalanta,NaN,NaN,4.200,3.600,1.850,NaN,NaN,NaN,...,0,0,3,2025,0.500000,0.866025,25,0.417194,0.908818,3


In [17]:
def compute_winstreak(df):
    
    years = df.Year.unique()
    df_lst = []    
    for year in years:
        
        year_df = df[df.Year == year]
        year_df['HomeWin'] = year_df.Result.replace([0, 1, 2], [0, 0, 1])
        year_df['AwayWin'] = year_df.Result.replace([0, 1, 2], [1, 0, 0])
        year_df['HomeWinStreak'] = None
        year_df['AwayWinStreak'] = None
        
        hometeams = year_df.HomeTeam.unique()
        awayteams = year_df.AwayTeam.unique()
        
        for team in hometeams:
            team_df = year_df[(year_df.HomeTeam == team)]
            team_df = team_df.sort_values(['Year', 'DayofYear'], ascending = (True, True))

            team_grouper = (team_df.HomeWin != team_df.HomeWin.shift()).cumsum()
            team_df['HomeWinStreak'] = team_df[['HomeWin']].groupby(team_grouper).cumsum()
            team_df.loc[team_df.HomeWinStreak >0, 'HomeWinStreak'] -= 1
            year_df.loc[team_df.index, 'HomeWinStreak'] = team_df.HomeWinStreak
            
        for team in awayteams:
            team_df = year_df[(year_df.AwayTeam == team)]
            team_df = team_df.sort_values(['Year', 'DayofYear'], ascending = (True, True))

            team_grouper = (team_df.AwayWin != team_df.AwayWin.shift()).cumsum()
            team_df['AwayWinStreak'] = team_df[['AwayWin']].groupby(team_grouper).cumsum()
            team_df.loc[team_df.AwayWinStreak >0, 'AwayWinStreak'] -= 1
            year_df.loc[team_df.index, 'AwayWinStreak'] = team_df.AwayWinStreak
            
        df_lst.append(year_df)
        
    return pd.concat(df_lst, axis = 0).drop(columns = ['HomeWin', 'AwayWin'])#,'DayofYear'])
df_noNa_div_date_lastMatch_streak = compute_winstreak(df_noNa_div_date_lastMatch)
df_noNa_div_date_lastMatch_streak

,HomeTeam,AwayTeam,FTHG,FTAG,B365H,B365D,B365A,WHH,WHD,WHA,...,Result,Year,Sin_Month,Cos_Month,DayofYear,Sin_Day,Cos_Day,Last Match Result,HomeWinStreak,AwayWinStreak
0,Auxerre,Nice,1.0,2.0,1.727,3.100,4.500,1.66,3.1,5.0,...,0,2003,-0.866025,-0.500000,214,-0.516062,-0.856551,3,0,0
1,Guingamp,Marseille,0.0,1.0,2.500,2.875,2.625,2.60,3.1,2.4,...,0,2003,-0.866025,-0.500000,214,-0.516062,-0.856551,3,0,0
2,Hamburg,Hannover,0.0,3.0,1.571,3.500,5.000,1.57,3.5,5.0,...,0,2003,-0.866025,-0.500000,214,-0.516062,-0.856551,3,0,0
3,Hertha,Werder Bremen,0.0,3.0,1.833,3.200,3.750,1.72,3.4,4.0,...,0,2003,-0.866025,-0.500000,214,-0.516062,-0.856551,3,0,0
4,Lens,Le Mans,0.0,0.0,1.571,3.250,5.500,1.66,3.3,4.5,...,0,2003,-0.866025,-0.500000,214,-0.516062,-0.856551,3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37308,Ath Madrid,Villarreal,NaN,NaN,1.670,4.000,5.000,NaN,NaN,NaN,...,3,2025,0.500000,0.866025,25,0.417194,0.908818,2,2,2
37309,Valladolid,Real Madrid,NaN,NaN,10.000,6.000,1.270,NaN,NaN,NaN,...,3,2025,0.500000,0.866025,25,0.417194,0.908818,0,2,2
37310,Mallorca,Betis,NaN,NaN,2.350,3.100,3.300,NaN,NaN,NaN,...,3,2025,0.500000,0.866025,25,0.417194,0.908818,0,2,2
37311,Como,Atalanta,NaN,NaN,4.200,3.600,1.850,NaN,NaN,NaN,...,3,2025,0.500000,0.866025,25,0.417194,0.908818,3,2,2


In [18]:
def compute_winstodate(df):
    
    years = df.Year.unique()
    df_lst = []    
    for year in years:
        
        year_df = df[df.Year == year]
        year_df['HomeWin'] = year_df.Result.replace([0, 1, 2], [0, 0, 1])
        year_df['AwayWin'] = year_df.Result.replace([0, 1, 2], [1, 0, 0])
        year_df['HomeWinsToDate'] = None
        year_df['AwayWinsToDate'] = None
        
        hometeams = year_df.HomeTeam.unique()
        awayteams = year_df.AwayTeam.unique()
        
        for team in hometeams:
            team_df = year_df[(year_df.HomeTeam == team)]
            team_df = team_df.sort_values(['Year', 'DayofYear'], ascending = (True, True))

            team_df['HomeWinsToDate'] = team_df.HomeWin.cumsum()
            year_df.loc[team_df.index, 'HomeWinsToDate'] = team_df.HomeWinsToDate
            
        for team in awayteams:
            team_df = year_df[(year_df.AwayTeam == team)]
            team_df = team_df.sort_values(['Year', 'DayofYear'], ascending = (True, True))
            
            team_df['AwayWinsToDate'] = team_df.AwayWin.cumsum()
            year_df.loc[team_df.index, 'AwayWinsToDate'] = team_df.AwayWinsToDate
            
        df_lst.append(year_df)
        
    return pd.concat(df_lst, axis = 0).drop(columns = ['HomeWin', 'AwayWin','DayofYear'])
df_noNa_div_date_lastMatch_streak_windate = compute_winstodate(df_noNa_div_date_lastMatch_streak)
df_noNa_div_date_lastMatch_streak_windate.drop(columns = ['HomeTeam', 'AwayTeam'], inplace = True)
df_noNa_div_date_lastMatch_streak_windate

,FTHG,FTAG,B365H,B365D,B365A,WHH,WHD,WHA,AHh,B365AHH,...,Year,Sin_Month,Cos_Month,Sin_Day,Cos_Day,Last Match Result,HomeWinStreak,AwayWinStreak,HomeWinsToDate,AwayWinsToDate
0,1.0,2.0,1.727,3.100,4.500,1.66,3.1,5.0,-0.75,2.050,...,2003,-0.866025,-0.500000,-0.516062,-0.856551,3,0,0,0,1
1,0.0,1.0,2.500,2.875,2.625,2.60,3.1,2.4,0.00,1.925,...,2003,-0.866025,-0.500000,-0.516062,-0.856551,3,0,0,0,1
2,0.0,3.0,1.571,3.500,5.000,1.57,3.5,5.0,-0.75,1.800,...,2003,-0.866025,-0.500000,-0.516062,-0.856551,3,0,0,0,1
3,0.0,3.0,1.833,3.200,3.750,1.72,3.4,4.0,-0.75,2.025,...,2003,-0.866025,-0.500000,-0.516062,-0.856551,3,0,0,0,1
4,0.0,0.0,1.571,3.250,5.500,1.66,3.3,4.5,-0.75,1.900,...,2003,-0.866025,-0.500000,-0.516062,-0.856551,3,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37308,NaN,NaN,1.670,4.000,5.000,NaN,NaN,NaN,-0.75,1.890,...,2025,0.500000,0.866025,0.417194,0.908818,2,2,2,3,3
37309,NaN,NaN,10.000,6.000,1.270,NaN,NaN,NaN,1.75,1.900,...,2025,0.500000,0.866025,0.417194,0.908818,0,2,2,4,3
37310,NaN,NaN,2.350,3.100,3.300,NaN,NaN,NaN,-0.25,2.000,...,2025,0.500000,0.866025,0.417194,0.908818,0,2,2,3,3
37311,NaN,NaN,4.200,3.600,1.850,NaN,NaN,NaN,0.50,2.020,...,2025,0.500000,0.866025,0.417194,0.908818,3,2,2,3,3


In [19]:
betting_feats = ['B365H', 'B365D', 'B365A']
betting_feats

['B365H', 'B365D', 'B365A']

In [20]:
def compute_meanodds(df, betting_feats):
    """
    """
    home_odds = []
    away_odds = []
    draw_odds = []
    for odd in betting_feats:
        odd_type = odd[-1]
        if odd_type == 'H':
            home_odds.append(odd)
        elif odd_type == 'A':
            away_odds.append(odd)
        else:
            draw_odds.append(odd)
    avg_home_odds = df[home_odds].mean(axis=1)
    avg_away_odds = df[away_odds].mean(axis=1)
    avg_draw_odds = df[draw_odds].mean(axis=1)
    
    ordered_cols = ['HomeOdds', 'AwayOdds', 'DrawOdds'] + df.columns.tolist()
    
    df['HomeOdds'] = avg_home_odds
    df['AwayOdds'] = avg_away_odds
    df['DrawOdds'] = avg_draw_odds
    
    return df[ordered_cols]
df_odd = compute_meanodds(df_noNa_div_date_lastMatch_streak_windate, betting_feats)
df_odd.drop(columns = ['WHH', 'WHD', 'WHA', 'HomeOdds', 'AwayOdds', 'DrawOdds', 'FTHG', 'FTAG'], inplace = True)
df_odd

,B365H,B365D,B365A,AHh,B365AHH,B365AHA,HomeTeamELO,AwayTeamELO,Div 0,Div 1,...,Year,Sin_Month,Cos_Month,Sin_Day,Cos_Day,Last Match Result,HomeWinStreak,AwayWinStreak,HomeWinsToDate,AwayWinsToDate
0,1.727,3.100,4.500,-0.75,2.050,1.850,1702.604858,1611.196045,0,0,...,2003,-0.866025,-0.500000,-0.516062,-0.856551,3,0,0,0,1
1,2.500,2.875,2.625,0.00,1.925,1.975,1685.016113,1665.625732,0,0,...,2003,-0.866025,-0.500000,-0.516062,-0.856551,3,0,0,0,1
2,1.571,3.500,5.000,-0.75,1.800,2.100,1718.566284,1649.805298,1,0,...,2003,-0.866025,-0.500000,-0.516062,-0.856551,3,0,0,0,1
3,1.833,3.200,3.750,-0.75,2.025,1.875,1719.916748,1692.120972,1,0,...,2003,-0.866025,-0.500000,-0.516062,-0.856551,3,0,0,0,1
4,1.571,3.250,5.500,-0.75,1.900,2.000,1697.354004,1539.958130,0,0,...,2003,-0.866025,-0.500000,-0.516062,-0.856551,3,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37308,1.670,4.000,5.000,-0.75,1.890,2.010,1854.542480,1730.981079,0,0,...,2025,0.500000,0.866025,0.417194,0.908818,2,2,2,3,3
37309,10.000,6.000,1.270,1.75,1.900,2.000,1551.154175,1954.102173,0,0,...,2025,0.500000,0.866025,0.417194,0.908818,0,2,2,4,3
37310,2.350,3.100,3.300,-0.25,2.000,1.900,1663.129639,1654.958984,0,0,...,2025,0.500000,0.866025,0.417194,0.908818,0,2,2,3,3
37311,4.200,3.600,1.850,0.50,2.020,1.880,1590.385498,1901.152222,0,0,...,2025,0.500000,0.866025,0.417194,0.908818,3,2,2,3,3


In [21]:
from sklearn.preprocessing import MinMaxScaler
minmax_scaler = MinMaxScaler()
df_odd.loc[:, ['Year']] = minmax_scaler.fit_transform(df_odd.loc[:, ['Year']])

from sklearn.preprocessing import StandardScaler
std_scaler = StandardScaler()
# to_scale = ['HomeWinStreak','AwayWinStreak','HomeWinsToDate', 'AwayWinsToDate', 'HomeTeamELO', 'AwayTeamELO', 'HomeOdds', 'AwayOdds', 'DrawOdds'] + betting_feats
to_scale = ['HomeTeamELO', 'AwayTeamELO'] + betting_feats

df_odd.loc[:, to_scale] = std_scaler.fit_transform(df_odd.loc[:, to_scale])
df_odd

,B365H,B365D,B365A,AHh,B365AHH,B365AHA,HomeTeamELO,AwayTeamELO,Div 0,Div 1,...,Year,Sin_Month,Cos_Month,Sin_Day,Cos_Day,Last Match Result,HomeWinStreak,AwayWinStreak,HomeWinsToDate,AwayWinsToDate
0,-0.509874,-0.624302,-0.016688,-0.75,2.050,1.850,0.098076,-0.701430,0,0,...,0.0,-0.866025,-0.500000,-0.516062,-0.856551,3,0,0,0,1
1,-0.074858,-0.813058,-0.533110,0.00,1.925,1.975,-0.055764,-0.225951,0,0,...,0.0,-0.866025,-0.500000,-0.516062,-0.856551,3,0,0,0,1
2,-0.597665,-0.288736,0.121025,-0.75,1.800,2.100,0.237683,-0.364153,1,0,...,0.0,-0.866025,-0.500000,-0.516062,-0.856551,3,0,0,0,1
3,-0.450222,-0.540411,-0.223257,-0.75,2.025,1.875,0.249495,0.005503,1,0,...,0.0,-0.866025,-0.500000,-0.516062,-0.856551,3,0,0,0,1
4,-0.597665,-0.498465,0.258737,-0.75,1.900,2.000,0.052150,-1.323740,0,0,...,0.0,-0.866025,-0.500000,-0.516062,-0.856551,3,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37308,-0.541952,0.130722,0.121025,-0.75,1.890,2.010,1.427002,0.344971,0,0,...,1.0,0.500000,0.866025,0.417194,0.908818,2,2,2,3,3
37309,4.145865,1.808553,-0.906311,1.75,1.900,2.000,-1.226591,2.294082,0,0,...,1.0,0.500000,0.866025,0.417194,0.908818,0,2,2,4,3
37310,-0.159273,-0.624302,-0.347198,-0.25,2.000,1.900,-0.247195,-0.319132,0,0,...,1.0,0.500000,0.866025,0.417194,0.908818,0,2,2,3,3
37311,0.881839,-0.204844,-0.746565,0.50,2.020,1.880,-0.883453,1.831529,0,0,...,1.0,0.500000,0.866025,0.417194,0.908818,3,2,2,3,3


In [22]:
# 提取预测数据
testing_row_number = df_noNa_new.shape[0]
df_testing_data = df_odd.iloc[-testing_row_number:].loc[:, df_odd.columns != 'Result']
df_testing_data

,B365H,B365D,B365A,AHh,B365AHH,B365AHA,HomeTeamELO,AwayTeamELO,Div 0,Div 1,...,Year,Sin_Month,Cos_Month,Sin_Day,Cos_Day,Last Match Result,HomeWinStreak,AwayWinStreak,HomeWinsToDate,AwayWinsToDate
37295,-0.102997,-0.624302,-0.429826,-0.25,2.08,1.82,-0.778161,-0.311353,0,0,...,1.0,0.5,0.866025,0.401488,0.915864,0,2,2,3,4
37296,-0.328102,-0.120953,-0.319655,-0.25,1.83,2.07,0.968027,0.825138,0,1,...,1.0,0.5,0.866025,0.417194,0.908818,0,2,2,4,4
37297,-0.541952,-0.079007,0.258737,-0.75,1.87,2.03,0.770353,0.131309,0,1,...,1.0,0.5,0.866025,0.417194,0.908818,0,2,2,4,3
37298,-0.857099,6.842045,2.875277,-2.50,1.87,2.03,2.723395,-0.820069,0,1,...,1.0,0.5,0.866025,0.417194,0.908818,3,2,2,3,4
37299,2.457576,1.179366,-0.870506,1.25,2.03,1.87,-1.224510,1.248463,0,1,...,1.0,0.5,0.866025,0.417194,0.908818,0,2,2,3,4
37300,2.176194,0.550179,-0.856735,1.25,1.83,2.07,-0.195007,2.541329,0,1,...,1.0,0.5,0.866025,0.417194,0.908818,0,2,2,3,4
37301,-0.384378,0.214613,-0.292113,-0.50,2.01,1.89,2.225430,1.464304,0,1,...,1.0,0.5,0.866025,0.417194,0.908818,0,2,2,4,3
37302,-0.440655,-0.204844,-0.154400,-0.50,1.91,1.99,-0.714311,-1.139963,1,0,...,1.0,0.5,0.866025,0.417194,0.908818,2,2,2,3,4
37303,-0.570090,0.407564,0.121025,-1.00,2.05,1.85,0.752289,-0.345841,1,0,...,1.0,0.5,0.866025,0.417194,0.908818,0,2,2,3,3
37304,2.738957,1.598824,-0.889786,1.50,2.00,1.90,-0.289135,1.930862,1,0,...,1.0,0.5,0.866025,0.417194,0.908818,2,2,2,3,3


In [23]:
### 加载模型
# model_name = 'sklearn_mlp_' + local_time + '.joblib'
model_name = './' + 'sklearn_mlp_2025_01_23_23_11_20.joblib'
mlp = load(model_name)
predict_val = mlp.predict(df_testing_data)
series_pre = pd.Series(predict_val, name='Predicted')
series_pre
# compare_result = pd.concat([series_pre, yTe.reset_index()], axis=1)
# compare_result

0     2
1     0
2     2
3     0
4     2
5     0
6     0
7     2
8     2
9     0
10    2
11    2
12    2
13    2
14    2
15    2
16    2
17    2
Name: Predicted, dtype: int64